In [18]:
import os
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline, EulerAncestralDiscreteScheduler
import torch
from PIL import Image
import matplotlib.pyplot as plt

# Directorio para guardar imágenes
output_dir = "./productos_generados"
os.makedirs(output_dir, exist_ok=True)

# Cargar scheduler
model_id = "stabilityai/stable-diffusion-2-1-base"
scheduler = EulerAncestralDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")

# Cargar pipeline de texto a imagen
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    scheduler=scheduler,
    torch_dtype=torch.float16,
)
pipe = pipe.to("cuda" if torch.cuda.is_available() else "cpu")

# Función para guardar imágenes
def save_image(image, filename):
    path = os.path.join(output_dir, filename)
    image.save(path)
    print(f"Imagen guardada en {path}")

# Función para generar imagen producto con prompt y negative prompt
def generar_imagen_producto(prompt, negative_prompt=None, guidance_scale=7.5, num_inference_steps=50, nombre_archivo="producto.png"):
    image = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        guidance_scale=guidance_scale,
        num_inference_steps=num_inference_steps,
    ).images[0]
    save_image(image, nombre_archivo)
    return image

# Prompts y negative prompts de ejemplo
prompt_1 = "A high-resolution product photo of modern running sneakers on a white background"
negative_prompt_1 = "blurry, low quality, watermark, text"

imagen_1 = generar_imagen_producto(prompt_1, negative_prompt=negative_prompt_1, nombre_archivo="zapatillas_running.png")

prompt_2 = "A luxury leather handbag with gold accents, studio lighting, white background"
negative_prompt_2 = "blurry, low quality, watermark, text"

imagen_2 = generar_imagen_producto(prompt_2, negative_prompt=negative_prompt_2, nombre_archivo="bolso_lujo.png")

# Cargar pipeline Image-to-Image
pipe_img2img = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_id,
    scheduler=scheduler,
    torch_dtype=torch.float16,
)
pipe_img2img = pipe_img2img.to("cuda" if torch.cuda.is_available() else "cpu")

# Función para generar variación de imagen con prompt y negative prompt
def generar_variacion_imagen(init_image, prompt, negative_prompt=None, strength=0.7, guidance_scale=7.5, num_inference_steps=50, nombre_archivo="variacion.png"):
    from PIL import Image
    if not isinstance(init_image, Image.Image):
        init_image = Image.fromarray(init_image)
    init_image = init_image.convert("RGB").resize((512, 512))
    
    image = pipe_img2img(
        prompt=prompt,
        negative_prompt=negative_prompt,
        image=init_image,
        strength=strength,
        guidance_scale=guidance_scale,
        num_inference_steps=num_inference_steps,
    ).images[0]
    
    save_image(image, nombre_archivo)
    return image

# Prompt y negative prompt para la variación
prompt_variacion = "Same running sneakers but with red stripes and white laces"
negative_prompt_variacion = "blurry, low quality, watermark, text"

imagen_variada = generar_variacion_imagen(imagen_1, prompt_variacion, negative_prompt=negative_prompt_variacion, nombre_archivo="zapatillas_rojas.png")

# Mostrar imágenes originales y variación
def mostrar_imagenes(imagenes, titulos=None):
    n = len(imagenes)
    plt.figure(figsize=(5*n,5))
    for i, im in enumerate(imagenes):
        plt.subplot(1, n, i+1)
        plt.imshow(im)
        plt.axis("off")
        if titulos:
            plt.title(titulos[i])
    plt.show()

mostrar_imagenes([imagen_1, imagen_variada], ["Zapatillas original", "Variación con rayas rojas"])
